# Creating DCASE 2025 Task #1 Dataset
Adapted from **TAU-urban-acoustic-scenes-2022-mobile-development dataset**

In [ ]:
import os
import pandas as pd
import numpy as np
import os
import shutil

In [ ]:
train_csv_set_path = "split25.csv"
test_csv_set_path = "test.csv"
all_data_set_path = "dataset.csv"
basepath = "/kaggle/input/tau-urban-acoustic-scenes-2022-mobile-development/TAU-urban-acoustic-scenes-2022-mobile-development.audio."
path_2 = "/TAU-urban-acoustic-scenes-2022-mobile-development/audio/"
labels = []
pathes = []
sources= []

# Define the list of classes
classes = [
    "airport",
    "shopping_mall",
    "metro_station",
    "street_pedestrian",
    "public_square",
    "street_traffic",
    "tram",
    "bus",
    "metro",
    "park"
]

all_df = pd.read_csv(all_data_set_path)
train_df = pd.read_csv(train_csv_set_path, sep='\t')
test_df = pd.read_csv(test_csv_set_path, sep='\t')

train_filenames = set(train_df['filename'])
test_filenames = set(test_df['filename'])

print(f"Train and Test sets have {len(train_filenames)} and {len(test_filenames)} items ")

destination_folder = ""
os.makedirs("/kaggle/working/dataset/train", exist_ok=True)
os.makedirs("/kaggle/working/dataset/test", exist_ok=True)

train_data = {
    "filename" : [],
    "label" : [],
    "device" : [],
}
test_data = {
    "filename" : [],
    "label" : [],
    "device" : [],
}


num_unmatched = 0

# Iterate over all rows in all_df
for index, row in all_df.iterrows():
    file_path = row['path']  # Path from all_df
    basename = os.path.basename(file_path) # Extract the file name
    file_name = "audio/" + basename 
    
    # Check if the file belongs to train or test
    if file_name in train_filenames:
        destination_folder = "/kaggle/working/dataset/train"
        train_data['filename'].append(basename)
        train_data['label'].append(classes[row['label']])
        train_data['device'].append(row['source'])
        train_filenames.discard(file_name)
    elif file_name in test_filenames:
        destination_folder = "/kaggle/working/dataset/test"
        test_data['filename'].append(basename)
        test_data['label'].append(classes[row['label']])
        test_data['device'].append(row['source'])
        test_filenames.discard(file_name)
    else:
        num_unmatched = num_unmatched + 1
        continue  # Skip if no match is found
    
    # Construct the full destination path
    destination_path = os.path.join(destination_folder, basename)
    
    # Copy the file to the appropriate folder
    # print(f"Copied: {file_path} -> {destination_path}")
    shutil.copy(file_path, destination_path)
    

    # if index > 40:
    #     break
    if index % 1000 == 0:
        print(f"{index}/{len(all_df)}")


# Create a DataFrame and save it as a CSV file
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)
train_df.to_csv("/kaggle/working/dataset/train.csv", index=False)
test_df.to_csv("/kaggle/working/dataset/test.csv", index=False)

print(f"{len(train_filenames)} and {len(test_filenames)} items left in Train and Test sets")
print("File organization completed.")
            